
<a href="http://landlab.github.io"><img style="float: left; width: 300px;" src="https://landlab.csdms.io/_static/landlab_logo.png"></a>

# Mastering River Flow Dynamics: From Theory to Simulation

<hr>
<small>For more Landlab tutorials, click here: <a href="https://landlab.readthedocs.io/en/latest/user_guide/tutorials.html">https://landlab.readthedocs.io/en/latest/user_guide/tutorials.html</a></small>
<hr>


## Overview

This notebook demonstrates how to simulate river flow dynamics using the Landlab library's RiverFlowDynamics component. We'll explore the core principles of water flow in open channels while implementing a computational model based on the depth-averaged 2D shallow water equations.

The RiverFlowDynamics component runs a semi-implicit, semi-Lagrangian finite-volume approximation to the depth-averaged 2D shallow-water equations developed by Casulli and Cheng (1992). This powerful approach allows us to accurately model complex flow patterns in rivers and channels.

## Setup and Imports

Let's start by importing the necessary libraries:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import clear_output
from tqdm import trange

from landlab import RasterModelGrid
from landlab.components import RiverFlowDynamics
from landlab.plot.imshow import imshow_grid

## Create Grid and Set Parameter Values

First, let's define our simulation parameters and create a rectangular grid for our flow dynamics calculations:

In [ ]:
# Grid dimensions
nRows = 20
nCols = 60
cellSize = 0.1  # meters

# Physical parameters
mannings_n = 0.012  # Manning's roughness coefficient, [s/m^(1/3)]
channel_slope = 0.01  # Channel slope [m/m]

# Simulation parameters
n_timesteps = 100  # Number of timesteps
dt = 0.1  # Timestep duration, [s]

Now let's create the grid:

In [ ]:
grid = RasterModelGrid((nRows, nCols), xy_spacing=(cellSize, cellSize))

## Setting up the Topography

Let's create a rectangular channel with a sloped bed. We'll set higher elevations on the sides to create channel walls:

In [ ]:
# Create topographic elevation field with a sloped channel
te = grid.add_zeros("topographic__elevation", at="node")
te += 0.059 - channel_slope * grid.x_of_node  # Sloped channel bed

# Create channel walls by raising the elevation on the sides
te[grid.y_of_node > 1.5] = 1.0  # North wall
te[grid.y_of_node < 0.5] = 1.0  # South wall

## Visualizing the Initial Topography

Let's visualize our channel topography using an enhanced plot:

In [ ]:
plt.figure(figsize=(12, 6))
im = imshow_grid(grid, "topographic__elevation", cmap="terrain")
plt.colorbar(im, label="Elevation (m)")
plt.title("Channel Topography", fontsize=14)
plt.show()

Let's also visualize a longitudinal profile along the channel centerline:

In [ ]:
# Extract the bed profile along the middle row
center_row = nRows // 2
middleBedProfile = np.reshape(te, (nRows, nCols))[center_row, :]

plt.figure(figsize=(12, 5))
plt.plot(np.arange(nCols) * cellSize, middleBedProfile, 'brown', linewidth=2)
plt.title("Longitudinal Profile of Channel Bed", fontsize=14)
plt.xlabel("Distance Along Channel (m)", fontsize=12)
plt.ylabel("Elevation (m)", fontsize=12)
plt.grid(True)
plt.show()

## Initializing Water Flow Fields

Before running the simulation, we need to set up the initial conditions for water depth, velocity, and water surface elevation:

In [ ]:
# Initially, channel is empty (water depth = 0)
h = grid.add_zeros("surface_water__depth", at="node")

# Initial water velocity is zero everywhere
vel = grid.add_zeros("surface_water__velocity", at="link")

# Initial water surface elevation equals the topographic elevation
wse = grid.add_zeros("surface_water__elevation", at="node")
wse += h + te

## Setting up Boundary Conditions

Now we'll define where water enters the domain and the corresponding water depth and velocity values at these entry points:

In [ ]:
# Define entry nodes at the left boundary where water will enter
fixed_entry_nodes = np.arange(300, 910, 60)  # Nodes along left edge within channel
fixed_entry_links = grid.links_at_node[fixed_entry_nodes][:, 0]  # Links connecting to these nodes

# Set water depth at entry nodes (0.5m)
entry_nodes_h_values = np.full(11, 0.5)  

# Set water velocity at entry links (0.45 m/s flowing into domain)
entry_links_vel_values = np.full(11, 0.45)

Let's visualize the inflow boundary conditions:

In [ ]:
plt.figure(figsize=(10, 6))

# Get y-coordinates of nodes at left edge
left_edge_y = grid.y_of_node[grid.nodes_at_left_edge]
left_edge_topo = te[grid.nodes_at_left_edge]

# Plot channel bottom
plt.plot(left_edge_y, left_edge_topo, 'brown', linewidth=2, label='Channel Bottom')

# Plot water surface at entry nodes
entry_y = grid.y_of_node[fixed_entry_nodes]
entry_topo = te[fixed_entry_nodes]
plt.plot(entry_y, entry_topo + entry_nodes_h_values, 'b-', linewidth=2, label='Water Surface')

# Fill between channel bottom and water surface
plt.fill_between(entry_y, entry_topo, entry_topo + entry_nodes_h_values, color='blue', alpha=0.3)

plt.title("Inflow Boundary Condition at Left Edge", fontsize=14)
plt.xlabel("Distance Across Channel (m)", fontsize=12)
plt.ylabel("Elevation (m)", fontsize=12)
plt.grid(True)
plt.legend()
plt.show()

## Initialize the River Flow Dynamics Component

Now that we've set up our grid, topography, and boundary conditions, we can initialize the RiverFlowDynamics component:

In [ ]:
rfd = RiverFlowDynamics(
    grid,
    dt=dt,
    mannings_n=mannings_n,
    fixed_entry_nodes=fixed_entry_nodes,
    fixed_entry_links=fixed_entry_links,
    entry_nodes_h_values=entry_nodes_h_values,
    entry_links_vel_values=entry_links_vel_values,
)

## Run the Simulation

Now we'll run the simulation for 100 timesteps (10 seconds) with visualization:

In [ ]:
# Set animation frequency
display_animation_freq = 5

for timestep in range(n_timesteps):
    rfd.run_one_step()
    
    if timestep % display_animation_freq == 0:
        clear_output(wait=True)  # Clear previous output
        plt.figure(figsize=(12, 6))
        im = imshow_grid(grid, "surface_water__depth", cmap="Blues")
        plt.colorbar(im, label="Water Depth (m)")
        plt.title(f"Water Depth at Time Step {timestep+1} (Time: {(timestep+1)*dt:.1f}s)", fontsize=14)
        plt.show()

## Analyze Final Results

Let's analyze the final state of our simulation by visualizing key hydraulic parameters:

### Final Water Depth and Surface Elevation

In [ ]:
# Create a figure with two subplots for water depth and surface elevation
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))

# Plot final water depth
plt.subplot(2, 1, 1)
im1 = imshow_grid(grid, "surface_water__depth", cmap="Blues")
plt.colorbar(im1, label="Depth (m)")
plt.title(f"Final Water Depth after {n_timesteps} Time Steps", fontsize=14)

# Plot final water surface elevation
plt.subplot(2, 1, 2)
im2 = imshow_grid(grid, "surface_water__elevation", cmap="terrain")
plt.colorbar(im2, label="Elevation (m)")
plt.title(f"Final Water Surface Elevation after {n_timesteps} Time Steps", fontsize=14)

plt.tight_layout()
plt.show()

### Longitudinal Profiles

In [ ]:
# Extract the centerline nodes
center_row = nRows // 2
center_nodes = np.arange(center_row * nCols, (center_row + 1) * nCols, dtype=int)

# Plot water depth along centerline
plt.figure(figsize=(12, 5))
plt.plot(grid.x_of_node[center_nodes], grid.at_node['surface_water__depth'][center_nodes], 
         'b-', linewidth=2, label='Water Depth')
plt.grid(True)
plt.xlabel('Distance along channel (m)', fontsize=12)
plt.ylabel('Water Depth (m)', fontsize=12)
plt.title('Longitudinal Profile of Water Depth along Channel Centerline', fontsize=14)
plt.legend()
plt.show()

# Plot water surface elevation and topography along centerline
plt.figure(figsize=(12, 5))
plt.plot(grid.x_of_node[center_nodes], grid.at_node['surface_water__elevation'][center_nodes], 
         'b-', linewidth=2, label='Water Surface')
plt.plot(grid.x_of_node[center_nodes], grid.at_node['topographic__elevation'][center_nodes], 
         'brown', linewidth=2, label='Channel Bottom')

# Fill between terrain and water surface where there's water
water_indices = grid.at_node['surface_water__depth'][center_nodes] > 0
plt.fill_between(grid.x_of_node[center_nodes][water_indices], 
                 grid.at_node['topographic__elevation'][center_nodes][water_indices],
                 grid.at_node['surface_water__elevation'][center_nodes][water_indices],
                 color='blue', alpha=0.3)

plt.grid(True)
plt.xlabel('Distance along channel (m)', fontsize=12)
plt.ylabel('Elevation (m)', fontsize=12)
plt.title('Longitudinal Profile of Water Surface and Channel Bottom', fontsize=14)
plt.legend()
plt.show()

### Flow Velocity Analysis

In [ ]:
# Extract velocity components
links_at_node = grid.links_at_node
velocity_at_links = grid.at_link["surface_water__velocity"]

# Calculate velocity magnitude at nodes (approximate)
velocity_magnitude = np.zeros(grid.number_of_nodes)
for i in range(grid.number_of_nodes):
    # Get valid links (those that are not -1)
    valid_links = links_at_node[i][links_at_node[i] >= 0]
    if len(valid_links) > 0:
        # Take the maximum velocity of connected links as an approximation
        velocity_magnitude[i] = np.max(np.abs(velocity_at_links[valid_links]))

# Add velocity magnitude field to grid
grid.at_node["velocity_magnitude"] = velocity_magnitude

# Plot velocity magnitude
plt.figure(figsize=(12, 6))
im = imshow_grid(grid, "velocity_magnitude", cmap="plasma")
plt.colorbar(im, label="Velocity Magnitude (m/s)")
plt.title(f"Flow Velocity Magnitude after {n_timesteps} Time Steps", fontsize=14)
plt.show()

# Plot velocity along centerline
plt.figure(figsize=(12, 5))
plt.plot(grid.x_of_node[center_nodes], velocity_magnitude[center_nodes], 
         'r-', linewidth=2, label='Flow Velocity')
plt.grid(True)
plt.xlabel('Distance along channel (m)', fontsize=12)
plt.ylabel('Velocity (m/s)', fontsize=12)
plt.title('Velocity Profile along Channel Centerline', fontsize=14)
plt.legend()
plt.show()

## Cross-sectional Analysis

In [ ]:
# Choose a cross-section near the middle of the channel
cross_section_x = nCols // 2
cross_section_nodes = np.array([cross_section_x + i * nCols for i in range(nRows)], dtype=int)

# Plot cross-section showing water and channel
plt.figure(figsize=(10, 6))

# Plot channel bottom
plt.plot(grid.y_of_node[cross_section_nodes], 
         grid.at_node['topographic__elevation'][cross_section_nodes], 
         'brown', linewidth=2, label='Channel Bottom')

# Plot water surface
plt.plot(grid.y_of_node[cross_section_nodes], 
         grid.at_node['surface_water__elevation'][cross_section_nodes], 
         'b-', linewidth=2, label='Water Surface')

# Fill between terrain and water surface where there's water
water_indices = grid.at_node['surface_water__depth'][cross_section_nodes] > 0
plt.fill_between(grid.y_of_node[cross_section_nodes][water_indices], 
                 grid.at_node['topographic__elevation'][cross_section_nodes][water_indices],
                 grid.at_node['surface_water__elevation'][cross_section_nodes][water_indices],
                 color='blue', alpha=0.3)

plt.grid(True)
plt.xlabel('Distance across channel (m)', fontsize=12)
plt.ylabel('Elevation (m)', fontsize=12)
plt.title(f'Channel Cross-section at x = {cross_section_x*cellSize:.1f} m', fontsize=14)
plt.legend()
plt.show()

## Interpretation of Results

Our simulation has captured several key aspects of river flow dynamics:

1. **Flow Development**: We observed how water progressively fills the channel, starting from the entry boundary and advancing downstream until reaching a quasi-steady state.

2. **Water Surface Profile**: The water surface shows a characteristic profile, with a gradual decrease in elevation downstream, corresponding to the energy gradient needed to overcome friction.

3. **Velocity Distribution**: The velocity field reveals how flow accelerates and decelerates in response to changes in channel geometry and water depth, showcasing the conservation of mass and momentum principles.

4. **Friction Effects**: Manning's roughness coefficient (n = 0.012) produces realistic flow resistance, resulting in a balance between gravitational forces and frictional resistance.

5. **Steady-State Flow**: After sufficient time steps, the flow approaches a steady-state condition with consistent depth and velocity patterns, characteristic of uniform flow in an open channel.

This simulation provides valuable insights into the fundamental principles of open-channel hydraulics and demonstrates the capability of the RiverFlowDynamics component to model complex flow phenomena with high fidelity.

-- --
### And that's it! 

You've successfully completed this tutorial on river flow dynamics modeling. You now understand how to set up, run, and analyze simulations of water flow in open channels using the RiverFlowDynamics component in Landlab. This knowledge forms the foundation for more complex simulations involving natural river systems, floodplains, and hydraulic structures.

-- --



### Click here for more <a href="https://landlab.csdms.io/tutorials/">Landlab tutorials</a>